In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
import stringer1
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline



In [2]:
df=pd.read_csv('new2.csv')

In [3]:
c

In [4]:
train=df.drop([1,7,11,17,22,27,34,38,43,46,52,57])
test=df.iloc[[1,7,11,17,22,27,34,38,43,46,52,57]]

In [5]:
train= train[pd.notnull(train['text'])]
test=test[pd.notnull(test['text'])]
print(train['target'].apply(lambda x: len(x.split(' '))).sum())

48


In [6]:
def print_plot(index):
    example = train[train.index == index][['text', 'target']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
#print_plot(10)

In [7]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
train['text'] = train['text'].apply(clean_text)
#print_plot(10)
k=open(file_name,'rb')
s=stringer1.extract(k)
y_pred=s.text

In [8]:
train['text'].apply(lambda x: len(x.split(' '))).sum()

108704

In [9]:
X = train.text
Y = train.target
x=test.text
y=test.target
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [10]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X, Y)


from sklearn.metrics import classification_report,confusion_matrix
y_pred1 = nb.predict(x)

print(accuracy_score(y_pred1, y))
#print(classification_report(y_test, y_pred,target_names=))

0.9166666666666666


In [11]:
print(classification_report(y, y_pred1))
print(confusion_matrix(y,y_pred1))

              precision    recall  f1-score   support

          cr       1.00      0.67      0.80         3
          fm       0.75      1.00      0.86         3
          lb       1.00      1.00      1.00         3
          ld       1.00      1.00      1.00         3

    accuracy                           0.92        12
   macro avg       0.94      0.92      0.91        12
weighted avg       0.94      0.92      0.91        12

[[2 1 0 0]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 0 3]]


In [12]:
y_pred

'labour laws'

In [13]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X, Y)


y_pred1 = sgd.predict(x)

print('accuracy %s' % accuracy_score(y_pred1, y))
print(classification_report(y, y_pred1))

accuracy 0.8333333333333334
              precision    recall  f1-score   support

          cr       1.00      0.33      0.50         3
          fm       0.75      1.00      0.86         3
          lb       1.00      1.00      1.00         3
          ld       0.75      1.00      0.86         3

    accuracy                           0.83        12
   macro avg       0.88      0.83      0.80        12
weighted avg       0.88      0.83      0.80        12



In [14]:
print(confusion_matrix(y,y_pred1))

[[1 1 0 1]
 [0 3 0 0]
 [0 0 3 0]
 [0 0 0 3]]
